In [1]:
import selenium
from bs4 import BeautifulSoup
import requests
import json 

In [2]:
Director = {
    "Name" : ""
}
Genre = {
    "Name" : ""
}
Star = {
    "Name" : "",
    "ImageUrl" : ""
}
Data = {
    "Title" : "",
    "ReleaseDate" : "",
    "Minute" : "",
    "ImageSm" : "",
    "Rating" : 0,
    "Description" : "",
    "Director" : "",
    "Genres" : [],
    "Stars" : [],
    "ImageLg" : "",
    "YoutubeLink" : ""
}

In [ ]:
with open('moviesnew.json', 'a') as outfile:
    url = "https://www.imdb.com/search/title/?title_type=feature&genres=sci-fi&sort=boxoffice_gross_us,desc&explore=genres"
    for counter in range(30):
        r = requests.get(url)
        soup = BeautifulSoup(r.content,"html.parser")
        print(url)
        listerList = soup.find_all("div", {"class":"lister-list"})
        listerListItem = listerList[0].find_all("div" , {"class" : "lister-item mode-advanced"})
        for movie in listerListItem:
            try :
                title = movie.find_all("h3" , {"class" : "lister-item-header"})[0].a.text
                print(title)
                release_date = movie.find_all("span" , {"class" : "lister-item-year text-muted unbold"})[0].text
                youtube_link = "https://www.youtube.com/results?search_query=" + title + " trailer"
                pInfos = movie.find_all("p" , {"class" : "text-muted"})
                runtime = pInfos[0].find_all("span" , {"class" : "runtime"})
                # time
                if  len(runtime) > 0 :
                    runtime = runtime[0].text
                else :
                    runtime = 0

                # genres
                genres = pInfos[0].find_all("span" , {"class" : "genre"})
                genreList = []
                if  len(genres) > 0 :
                    genres = genres[0].text
                    genres = genres.split(",")
                    for g in genres :
                        genreList.append(g.replace(" " , "").replace("\n",""))
                else :
                    genres = ""

                # rating
                rating = movie.find_all("div" , {"class" : "inline-block ratings-imdb-rating"})
                if  len(rating) > 0 :
                    rating = rating[0].strong.text
                else :
                    rating = 0

                # description
                description = pInfos[1].text

                # director and stars
                pNoClass = movie.find_all("p" , {"class" : ""})[0]
                pNoClass = pNoClass.find_all("a")
                index = 0
                stars = []
                urls = []
                director = ""
                for i in pNoClass :
                    if index == 0 :
                        director = i.text
                        index+= 1
                    else :
                        star_name = i.text
                        star_link =  "https://www.imdb.com" + i['href']
                        requestForStarDetail = requests.get(star_link)
                        soupStarDetail = BeautifulSoup(requestForStarDetail.content,"html.parser")
                        star_poster = soupStarDetail.find_all("div" , {"class" : "poster-hero-container"})
                        poster = star_poster[0].img['src']
                        stars.append({"Name" : star_name , "ImageUrl" : poster})

                poster = movie.find_all("div" , {"class" : "lister-item-image float-left"})[0]

                # movie detail
                movie_detail_link = "https://www.imdb.com/" + poster.a['href']
                requestForMovieDetail = requests.get(movie_detail_link)
                soupMovieDetail = BeautifulSoup(requestForMovieDetail.content,"html.parser")

                poster_lg = soupMovieDetail.find_all("div",{"class","sc-a658a417-1 bmkYoJ"})
                poster_lg_link = "https://www.imdb.com/" + poster_lg[0].a['href']

                requestForPoster = requests.get(poster_lg_link)
                soupForPoster = BeautifulSoup(requestForPoster.content,"html.parser")

                image_lg = soupForPoster.find_all("div",{"class":"sc-7c0a9e7c-2 bkptFa"})
                image_lg = image_lg[0].img['src']

                Data["Director"] = director
                Data["ReleaseDate"] = release_date.replace("(","").replace(")","")
                Data["Title"] = title
                Data["Minute"] = runtime
                Data["YoutubeLink"] = youtube_link
                Data["Genres"] = genreList
                Data["Rating"] = rating
                Data["Stars"] = stars
                Data["ImageSm"] = ""
                Data["ImageLg"] = image_lg
                Data["Description"] = description
                
                json_object = json.dumps(Data , indent = 4) 
                outfile.write(json_object)
                outfile.write(",\n")
            
            except : 
                continue
        
        next_page = soup.find_all("div" , {"class" : "desc"})[0]
        next_page = next_page.find_all("a" , {"class" :"lister-page-next next-page"})[0]['href']

        url = "https://www.imdb.com" + next_page
    outfile.close()